In [ ]:
from pprint import pprint as pp
from typing import Dict, List, Tuple

import random
import csv
import time

# Building Knowledge Graph

## Extract Data from CSV

In [ ]:
def get_csv_data(filepath, handle_row_func):
    data = dict()
    with open(filepath) as file:
        next(file)
        rows = csv.reader(file, delimiter=",")
        for row in rows:
            handle_row_func(data, row)
    return data

In [ ]:
def handle_csv_kanji_func(data, row):
    kanji,*meanings = row
    if len(meanings) >= 2:
        meanings = ",".join(meanings)
    else:
        meanings = meanings[0]
    meanings = meanings.split(":")
    meanings = meanings[0]
    data[kanji] = meanings

In [ ]:
data_kanji = get_csv_data("dataset/s5_kanjis_output.csv", handle_csv_kanji_func)
print("len(data_kanji) = ", len(data_kanji))
# pp(data_kanji)

In [ ]:
def handle_csv_radical_func(data, row):
    radical,meaning,_ = row
    data[radical] = meaning

In [ ]:
data_radical = get_csv_data("dataset/s7_nodes_radical_meaning.csv", handle_csv_radical_func)
print("len(data_radical) = ", len(data_radical))
# pp(data_radical)

In [ ]:
def handle_csv_edges_func(data, row):
    kanji,radical_list = row
    data[kanji] = radical_list.split(':')

In [ ]:
data_edges = get_csv_data("dataset/s7_edges_kanji_radical.csv", handle_csv_edges_func)
print("len(data_edges) = ", len(data_edges))
# pp(data_edges)

## Data Structure

### Node Manager

In [ ]:
def get_key(symbol, dtype):
    return f"{symbol}-{dtype}"

In [ ]:
node_kanji = {
    get_key(symbol, 'kanji'): {
        'symbol' : symbol,
        'meaning': meaning,
        'visual' : f"{symbol}\n{meaning}",
        'color'  : 'red',
    } for symbol, meaning in data_kanji.items()}

In [ ]:
node_radical = {
    get_key(symbol, 'radical'): {
        'symbol' : symbol,
        'meaning': meaning,
        'visual' : f"{symbol}\n{meaning}",
        'color'  : 'yellow',
    } for symbol, meaning in data_radical.items()}

In [ ]:
full_node = {**node_radical, **node_kanji}
# pp(list(full_node.keys()))

### Edge Manager

In [ ]:
def get_graph_edge(data_edges):
    edges = []
    for kanji, radicals in data_edges.items():
        for r in radicals:
            edges.append( (f"{kanji}-kanji", f"{r}-radical") )
    return edges

full_edges = get_graph_edge(data_edges)
# full_edges

## Graph

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

G.add_nodes_from(full_node.items())
G.add_edges_from(full_edges)

### Graph Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm

# Reference: https://albertauyeung.github.io/2020/03/15/matplotlib-cjk-fonts.html
[f for f in fm.fontManager.ttflist if 'CJK JP' in f.name]

In [ ]:
def visualize_graph(
    Graph: nx.Graph, 
    figsize: tuple=(7,7), 
    color_map: List[str]=None, 
    node_size: int=3000, 
    with_labels: bool=True) -> None:
    
    if color_map == None:
        color_map = [Graph.nodes[n]["color"] for n in Graph] 
    else:
        color_map = color_map
    
    plt.figure(1,figsize=figsize) 
    
    labels = nx.get_node_attributes(Graph, 'visual')
    
    nx.draw_kamada_kawai(Graph, 
                         node_color=color_map, 
                         with_labels=with_labels,
                         labels=labels,
                         node_size=node_size, 
                         font_size=20,
                         font_family="Noto Serif CJK JP")
    plt.show()

In [ ]:
visualize_graph(
    Graph       = G.subgraph(random.sample(G.nodes, 300)),
    node_size   = 100,
    with_labels = None,
)

In [ ]:
def get_sg_kanji_with() -> nx.Graph:
    sg = nx.Graph()
    p = '痘-kanji'
    radicals = [n for n in G.neighbors(p)]
    sg.add_nodes_from([(p, G.nodes[p])] + [(r, G.nodes[r]) for r in radicals])
    sg.add_edges_from([(p, rp) for rp in radicals])
    return sg

In [ ]:
visualize_graph(
    Graph       = get_sg_kanji_with(),
    node_size   = 2000,
    with_labels = True,
)

### Exploratory Data Analysis

In [ ]:
print(nx.info(G))

In [ ]:
# https://networkx.org/documentation/stable/reference/algorithms/component.html

print('number of connected components: ', nx.number_connected_components(G))

In [ ]:
# https://networkx.org/documentation/stable/reference/algorithms/isolates.html
# EDA + Preprocessing: Removing Isolated Nodes

print('number of isolated: ', nx.number_of_isolates(G))
isolated_nodes = [n for n in nx.isolates(G)]
pp(isolated_nodes)
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
print(nx.info(G))

print('number of connected components: ', nx.number_connected_components(G))

In [ ]:
visualize_graph(
    Graph       = G.subgraph(random.sample(G.nodes, 300)),
    node_size   = 100,
    with_labels = None,
)

# Querying Knowledge Graph

## Data Structure

In [ ]:
def generate_graph(nodes: List) -> nx.Graph:
    R = nx.Graph()
    R.add_nodes_from([(n, G.nodes[n]) for n in nodes])
    R.add_edges_from(nx.utils.pairwise(nodes))
    return R

In [ ]:
def get_node_color_result(kinputs, koutputs, union_result):
    G = union_result
    color_map = []
    for n in union_result:
        if n in kinputs:
            color_map.append("green")
        elif n in koutputs:
            color_map.append("blue")
        else:
            color_map.append(G.nodes[n]["color"])
    return color_map

In [ ]:
kin = '語-kanji'  # say
kout = '誕-kanji' # lie 

result_shortest_path = nx.shortest_path(G, source=kin, target=kout)
result = generate_graph(result_shortest_path)

visualize_graph(
    Graph=result, 
    color_map=get_node_color_result([kin], [kout], result), 
    figsize=(6,6)
)

## Algorithm

### Brute Force Algorithm

In [ ]:
def find_path_bf(G: nx.Graph, MOrig: List, MDest: List) -> nx.Graph:
    
    result = []
    
    for kin in MOrig:
        for kout in MDest:
            print(kin, kout)
            sp_raw = nx.dijkstra_path(G, source=kin, target=kout)
            sp_graph = generate_graph(sp_raw)
            
            result.append(sp_graph)
    
    
    return nx.compose_all(result)        

### Astar Algorithm

In [ ]:
def find_path_astar(G: nx.Graph, MOrig: List, MDest: List, heuristic_func) -> nx.Graph:
    
    result = []
    
    for kin in MOrig:
        for kout in MDest:
            sp_raw = nx.astar_path(G, source=kin, target=kout, heuristic=heuristic_func)
            sp_graph = generate_graph(sp_raw)
            
            result.append(sp_graph)
    
    
    return nx.compose_all(result)        

#### Heuristic

In [ ]:
def common_neighbor(u, v):
    return len(list(nx.common_neighbors(G, u, v)))

In [ ]:
def jaccard_function(u, v):
    union_size = len(set(G[u]) | set(G[v])) # union neighbor
    if union_size == 0:
        return 0
    return len(list(nx.common_neighbors(G, u, v))) / union_size

### Steiner Tree

In [ ]:
all_pairs_dijkstra = nx.all_pairs_dijkstra(G, weight='weight')

In [ ]:
def metric_closure(G, weight="weight"):
    M = nx.Graph()

    Gnodes = set(G)

    # check for connected graph while processing first node
    all_paths_iter = all_pairs_dijkstra
    u, (distance, path) = next(all_paths_iter)
    if Gnodes - set(distance):
        msg = "G is not a connected graph. metric_closure is not defined."
        raise nx.NetworkXError(msg)
    Gnodes.remove(u)
    for v in Gnodes:
        M.add_edge(u, v, distance=distance[v], path=path[v])

    # first node done -- now process the rest
    for u, (distance, path) in all_paths_iter:
        Gnodes.remove(u)
        for v in Gnodes:
            M.add_edge(u, v, distance=distance[v], path=path[v])

    return M

In [ ]:
def steiner_tree(G, terminal_nodes, weight="weight"):
    # H is the subgraph induced by terminal_nodes in the metric closure M of G.
    M = metric_closure(G, weight=weight)
    H = M.subgraph(terminal_nodes)
    # Use the 'distance' attribute of each edge provided by M.
    mst_edges = nx.minimum_spanning_edges(H, weight="distance", data=True)
    # Create an iterator over each edge in each shortest path; repeats are okay
    edges = chain.from_iterable(pairwise(d["path"]) for u, v, d in mst_edges)
    # For multigraph we should add the minimal weight edge keys
    if G.is_multigraph():
        edges = (
            (u, v, min(G[u][v], key=lambda k: G[u][v][k][weight])) for u, v in edges
        )
    T = G.edge_subgraph(edges)
    return T

In [ ]:
def find_path_steiner(G: nx.Graph, MOrig: List, MDest: List) -> nx.Graph:
    
    return nx.steiner_tree(G, MOrig + MDest)     

In [ ]:
# result = find_path_steiner(MOrig, MDest)
# result.nodes()

# Experiment

## Test Case

In [ ]:
from kanji_lists import JLPT, KYOIKU
# https://github.com/ffe4/kanji-lists

In [ ]:
test_cases_raw = {
    'N5 to N4': {
        'MOrig': JLPT.N5,
        'MDest': JLPT.N4,
    },
    'N4 to N3': {
        'MOrig': JLPT.N4,
        'MDest': JLPT.N3,
    },
    'N3 to N2': {
        'MOrig': JLPT.N3,
        'MDest': JLPT.N2,
    },
    'N2 to N1': {
        'MOrig': JLPT.N2,
        'MDest': JLPT.N1,
    },
    'G1 to G2': {
        'MOrig': KYOIKU.GRADE1,
        'MDest': KYOIKU.GRADE2,
    },
    'G2 to G3': {
        'MOrig': KYOIKU.GRADE2,
        'MDest': KYOIKU.GRADE3,
    },
    'G3 to G4': {
        'MOrig': KYOIKU.GRADE3,
        'MDest': KYOIKU.GRADE4,
    },
    'G4 to G5': {
        'MOrig': KYOIKU.GRADE4,
        'MDest': KYOIKU.GRADE5,
    },
    'G5 to G6': {
        'MOrig': KYOIKU.GRADE5,
        'MDest': KYOIKU.GRADE6,
    },
}

In [ ]:
def tc_filter_in_graph(kanji_list):
    
    tc_filtered = []
    for k in kanji_list:

        tk = get_key(k, 'kanji')
        
        if tk in G:
            tc_filtered.append(tk)
            continue
        
        tr = get_key(k, 'radical')
        if tr in G:
            tc_filtered.append(tr)
            continue
            
    return tc_filtered

In [ ]:
def filter_test_cases_raw(test_cases_raw):
    test_cases_clean = {}
    for tc_name, tc in test_cases_raw.items():

        MOrig = tc_filter_in_graph(tc['MOrig'])
        MDest = tc_filter_in_graph(tc['MDest'])

        test_cases_clean[tc_name] = {
            'MOrig': MOrig,
            'MDest': MDest,
        }

    return test_cases_clean

In [ ]:
test_cases_clean = filter_test_cases_raw(test_cases_raw)

In [ ]:
def find_path(G: nx.Graph, MOrig: List, MDest: List, method='brute_force') -> nx.Graph:
    if method == 'brute_force':
        return find_path_bf(G, MOrig, MDest)
    elif method == 'steiner_tree':
        return find_path_steiner(G, MOrig, MDest)
    elif method == 'astar_common_neighbor':
        return find_path_astar(G, MOrig, MDest, common_neighbor)
    elif method == 'astar_jaccard':
        return find_path_astar(G, MOrig, MDest, jaccard_function)
    elif method == 'astar_0':
        return find_path_astar(G, MOrig, MDest, lambda x, y: 0)
    else:
        raise ValueError(f"method {method} is not valid")

## Testing

In [ ]:
def get_result_accuracy(G0, Gt):
    common_nodes = len(G0.nodes() & Gt.nodes())
    G0_nodes     = len(G0.nodes())
    
    return common_nodes / G0_nodes

In [ ]:
def get_results_kanjigen(G: nx.Graph, algo_list: List, test_cases_clean: dict):
    results = {tc_name: dict() for tc_name in test_cases_clean}

    for tc_name, tc in test_cases_clean.items():
        
        if tc_name != 'N5 to N4':
            continue
        
        MOrig = tc['MOrig'][:3]
        MDest = tc['MDest'][:3]
        
        # get time
        for algo in algo_list:
            
            print(f"algo: {algo}")
            print(f"tc: {tc_name} of {len(test_cases_clean)}")
            start  = time.time()
            graph  = find_path(G, MOrig, MDest, algo)
            end    = time.time()
            print("finish")
            print("==============")
            
            results[tc_name][algo] = {'graph': graph, 'time': (end - start)}
        
        # get accuracy
        
        G0 = results[tc_name]['brute_force']['graph']
        for algo in [algo for algo in algo_list if algo != 'brute_force']:
            
            Gt = results[tc_name][algo]['graph']
            
            results[tc_name][algo]["accuracy"] = get_result_accuracy(G0, Gt)
    
    return results

In [ ]:
algo_list = ['brute_force', 'astar_common_neighbor', 'astar_jaccard', 'astar_0']
results = get_results_kanjigen(G, algo_list, test_cases_clean)

In [ ]:
pp(results)

# Analysis

In [188]:
import pandas as pd

In [189]:
pd

<module 'pandas' from '/home/ifkarsyah/Documents/ITB/kanjigen/kanjigen-be/venv/lib/python3.8/site-packages/pandas/__init__.py'>